In [1]:
N = 1
T1 = 4
T2 = 10
T3 = 10
PROC_NUM = 3
QUEUE_SIZE = 16
MAX_STATE = PROC_NUM + QUEUE_SIZE
MU = 1 / T1
NU = 1 / T2

In [2]:
def get_lambda(i):
    return N

In [3]:
def get_mu(i):
    if i <= PROC_NUM:
        return i * MU
    else:
        return PROC_NUM * MU + (i - PROC_NUM) * NU

In [4]:
def state_prob(i):
    prob = 1;
    if i == 0:
        for i in range(1, MAX_STATE + 1):
            prob += state_prob(i)
        prob = 1 / prob
    else:
        prob = 1
        for i in range(1, i + 1):
            prob *= get_lambda(i) / get_mu(i)
    return prob

In [5]:
sum = 0
states = dict()
for i in range(0, MAX_STATE + 1):
    if i == 0:
        states[0] = state_prob(0)
        sum += states[0]
    else:
        states[i] = states[0] * state_prob(i)
        sum += states[i]
    print('state prob {:2d} = {:.5f}'.format(i, states[i]))
print('{:>13s} = {:.5f}'.format('sum', sum))

state prob  0 = 0.01007
state prob  1 = 0.04027
state prob  2 = 0.08053
state prob  3 = 0.10738
state prob  4 = 0.12633
state prob  5 = 0.13298
state prob  6 = 0.12665
state prob  7 = 0.11013
state prob  8 = 0.08810
state prob  9 = 0.06526
state prob 10 = 0.04501
state prob 11 = 0.02904
state prob 12 = 0.01760
state prob 13 = 0.01006
state prob 14 = 0.00544
state prob 15 = 0.00279
state prob 16 = 0.00136
state prob 17 = 0.00063
state prob 18 = 0.00028
state prob 19 = 0.00012
          sum = 1.00000


Среднее количество занятых работников рассчитаем сумму произведений вероятностей состояний на количество занятых работников в них:

In [6]:
def busy_count(i):
    return i if i <= PROC_NUM else PROC_NUM

In [7]:
busy_cnt = 0
for i in range(0, MAX_STATE + 1):
    busy_cnt += states[i] * busy_count(i)
print('{} = {:.5f}'.format('buys_cnt', busy_cnt))

buys_cnt = 2.80873


Средняя длина очереди считается аналогично: сумма произведений вероятностей состояний на длину очереди в них:

In [8]:
def queue_length(i):
    return 0 if i <= PROC_NUM else i - PROC_NUM

In [9]:
queue_len = 0
for i in range(0, MAX_STATE + 1):
    queue_len += states[i] * queue_length(i)
print('{} = {:.5f}'.format('queue_len', queue_len))

queue_len = 2.97698


Вероятность отказа из-за заполненной очереди равна вероятности состояния 19:

In [10]:
reject_prob = states[MAX_STATE]
print('{} = {:.5f}'.format('reject_prob', reject_prob))

reject_prob = 0.00012


Вероятность ухода из очереди  отношение интенсивности ухода к интенсивности поступления заявок, умноженное на среднюю длину очереди:

In [11]:
leave_prob = NU / N * queue_len
print('{} = {:.5f}'.format('leave_prob', leave_prob))

leave_prob = 0.29770


Успешное обслуживание производится при отсутствии отказа при поступлении и ухода из очереди. Перемножив вероятности того, что эти события не наступят:

In [12]:
success_prob = (1 - leave_prob) * (1 - reject_prob)
print('{} = {:.5f}'.format('success_prob', success_prob))

success_prob = 0.70222


Вычислим среднее кол-во человек в системе как сумму средней длины очереди и занятых поваров:

In [13]:
mean_cnt = queue_len + busy_cnt
print('{} = {:.5f}'.format('mean_cnt', mean_cnt))

mean_cnt = 5.78571


Согласно закону Литтла среднее время нахождения в системе равно $t_c = \dfrac{j}{\lambda} + T$, где $T$ - время обеда, умноженное на вероятность успешного обслуживания:

In [14]:
mean_time = mean_cnt / N + T3 * success_prob
print('{} = {:.5f}'.format('mean_time', mean_time))

mean_time = 12.80789
